In [69]:
import networkx as nx
import pandas as pd
import numpy as np
import json
import pickle 
import matplotlib.pyplot as plt

# needed only if running this as a single page app
#external_stylesheets = [dbc.themes.LUX]

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# change to app.layout if running as single page app instead

graph_style = {"maxHeight": "auto", "align":"center", "margin": "auto"}

# Read data
df = pd.read_csv('HP_enriched_character_df.csv')

with open("G_exploratory.pickle", 'rb') as f:
    G = pickle.load(f)

with open("GCC_exploratory.pickle", 'rb') as f:
    GCC = pickle.load(f)

## Get partition

In [70]:
def get_partition(G, attribute, att_list):
    node_partition = {}
    for att in att_list:
        selected_nodes = [n for n,v in G.nodes(data = True) if v[attribute] == att]
        node_partition[att] = selected_nodes
        
    return node_partition

In [71]:
community_key = 'species'
communities = list(pd.unique(df[community_key]))

partition = get_partition(GCC, community_key, communities)


## Visualize

In [72]:
import random

In [73]:
def get_community_colors(communities):
    if "Gryffindor" in communities: 
        houses = True
    else:
        houses = False

    if houses == False: 
        col_map = {}
        for com in communities:
            col_map[com] = '#%06X' % random.randint(0, 0xFFFFFF) #np.random.randint(256)
        return col_map
    else:
        house_col_map = {'Hufflepuff': "#FFDB00",
           'Slytherin': '#1A472A',
           'Gryffindor': '#AE0001',
           'Ravenclaw':'#222F5B',
           'Thunderbird':'grey',
           'Pukwudgie':'lightgrey',
           'Horned':'black',
           'Wampus':'darkgrey', 
           'unknown': 'whitesmoke'}
        return house_col_map

colors = get_community_colors(communities)
#colors

In [74]:
forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=False,  # Dissuade hubs
                        linLogMode = False,  
                        adjustSizes = False,
                        #edgeWeightInfluence = 0.1,

                        # Performance
                        jitterTolerance = 1.0,  # Tolerance
                        barnesHutOptimize = True,
                        barnesHutTheta = 1.2,
                        multiThreaded = False,

                        # Tuning
                        scalingRatio=0.0001,
                        strongGravityMode=False,
                        gravity=2, # attracts nodes to the center

                        # Log
                        verbose=True)

NameError: name 'ForceAtlas2' is not defined

In [75]:
# get node size
sizes = list(nx.get_node_attributes(GCC,'size').values())

count = 0
fig = plt.figure(figsize = (15,15))
fig.patch.set_facecolor('silver')

# get positions
#positions = forceatlas2.forceatlas2_networkx_layout(GCC, pos = None, iterations = 1000)

for com in communities:
    count = count + 1.
    list_nodes = [node for node in GCC.nodes() if node in partition[com]]
    weights = [GCC.nodes(data=True)[node]['size'] for node in list_nodes]
    print(colors[com])
    nx.draw_networkx_nodes(GCC, positions, list_nodes, node_size = weights, node_color=colors[com])

nx.draw_networkx_edges(GCC, positions, edge_color="gray", alpha=0.1)
plt.axis('off')
plt.show()

#631845


NameError: name 'positions' is not defined

<Figure size 1080x1080 with 0 Axes>

In [76]:
com_dict = []
for key,val in partition.items():
    com_dict.append(set(val))

In [95]:
communities

['unknown',
 'Human',
 'Amortality',
 'Owl',
 'Hag',
 'Banshee',
 'Dragon',
 'Veela',
 'Half-breed',
 'Doxy',
 'Part elf',
 'Fairy',
 'Giant',
 'Werewolf',
 'Goblin',
 'House-elf',
 'Mouse',
 'Zouwu',
 'Dog',
 'Bowtruckle',
 'Selkie',
 'Kneazle',
 'Acromantula',
 'Cat',
 'Donkey',
 'Merpeople',
 'Vampire',
 'Basilisk',
 'Chimaera',
 'Sphinx',
 'Occamy',
 'Thestral',
 'Chicken',
 'Sheep',
 'Spider',
 'Peacock',
 'Chupacabra',
 'Pygmy Puff',
 'Centaur',
 'Fruit bat',
 'Rabbit',
 'Boa constrictor',
 'Wolf',
 'Hippogriff',
 'Quintaped',
 'Toad',
 'Demiguise',
 'Shrunken head',
 'Ghost',
 'Phoenix',
 'Three-headed dog',
 'Thunderbird',
 'Gorilla',
 'Poltergeist',
 'Augurey',
 'Crab',
 'Imp',
 'Knarl',
 'Manticore',
 'Niffler',
 'Red Cap',
 'Crup',
 'Frog',
 'Wampus cat',
 'Ghoul',
 'Pukwudgie',
 'Abraxan',
 'Ferret',
 'Budgerigar',
 'Pony',
 'Bird',
 'Parrot',
 'Tortoise',
 'Mooncalf',
 'Troll',
 'Fox',
 'Rat',
 'Snake',
 'Goat',
 'Giant Squid',
 'Tarantula',
 'Kelpie',
 'Adder',
 'Newt',
 

In [96]:
GCC.nodes(data=True)['Fleur Delacour']

{'species': 'Half-Veela',
 'gender': 'Female',
 'house': 'unknown',
 'blood': 'unknown',
 'color': 'grey',
 'in_degree': 77,
 'out_degree': 52,
 'degree': 129,
 'size': 645,
 'pos': (0.3937368033074402, -0.054132066378430696)}

In [87]:
inv_partition = {l: k for k, v in partition.items() for l in v}


In [88]:
# Measure quality of partition
import networkx.algorithms.community as nx_comm

import community 
mod = community.modularity(inv_partition, GCC)

#nx_comm.modularity(GCC, com_dict) 

KeyError: 'Fleur Delacour'

In [78]:
def get_modularity(G, partition):
    n_c = len(partition)
    L = G.number_of_edges()
    M = 0

    for com in partition.keys():
        A = G.subgraph(partition[com])
        k_c = A.number_of_nodes()
        L_c = A.number_of_edges()
        M += (L_c/L) - (k_c/(2*L))**2

    return M

modularity = get_modularity(GCC, partition) 
print('Modularity for the partition is {}'.format(modularity))

Modularity for the partition is 0.79979390736417


In [79]:
nx_comm.modularity(GCC, communities)

NotAPartition: Graph with 2747 nodes and 13198 edges is not a valid partition of the graph ['unknown', 'Human', 'Amortality', 'Owl', 'Hag', 'Banshee', 'Dragon', 'Veela', 'Half-breed', 'Doxy', 'Part elf', 'Fairy', 'Giant', 'Werewolf', 'Goblin', 'House-elf', 'Mouse', 'Zouwu', 'Dog', 'Bowtruckle', 'Selkie', 'Kneazle', 'Acromantula', 'Cat', 'Donkey', 'Merpeople', 'Vampire', 'Basilisk', 'Chimaera', 'Sphinx', 'Occamy', 'Thestral', 'Chicken', 'Sheep', 'Spider', 'Peacock', 'Chupacabra', 'Pygmy Puff', 'Centaur', 'Fruit bat', 'Rabbit', 'Boa constrictor', 'Wolf', 'Hippogriff', 'Quintaped', 'Toad', 'Demiguise', 'Shrunken head', 'Ghost', 'Phoenix', 'Three-headed dog', 'Thunderbird', 'Gorilla', 'Poltergeist', 'Augurey', 'Crab', 'Imp', 'Knarl', 'Manticore', 'Niffler', 'Red Cap', 'Crup', 'Frog', 'Wampus cat', 'Ghoul', 'Pukwudgie', 'Abraxan', 'Ferret', 'Budgerigar', 'Pony', 'Bird', 'Parrot', 'Tortoise', 'Mooncalf', 'Troll', 'Fox', 'Rat', 'Snake', 'Goat', 'Giant Squid', 'Tarantula', 'Kelpie', 'Adder', '

In [147]:
# Measure quality of partition
import networkx.algorithms.community as nx_comm

def quality_of_partition(G, partition):
    #coverage = nx.coverage(G, partition) #Returns the coverage of a partition.
    modularity = nx_comm.modularity(G, communities) #Returns the modularity of the given partition of the graph.
    #partition_quality = partition_quality(G, partition)# Returns the coverage and performance of a partition of G.
    #performance = performance(G, partition)
    
    metrics = {"coverage":coverage,"modularity":modularity, "partition_quality":partition_quality,"performance":performance  }
    return metrics

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=160b1b0f-c982-4c5c-bddd-69f07a3fe7a4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>